In [2]:
import csv
import re
import nltk
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
import re

In [3]:

# Input and output file paths
filename_in = 'C:/Users/ingeb/OneDrive - Danmarks Tekniske Universitet/Semester 1/Intelligent Systems 2024/3-ugers kursus/Democrats/SenatorCantwelltweets_clean_P.csv'
filename_out ='C:/Users/ingeb/OneDrive - Danmarks Tekniske Universitet/Semester 1/Intelligent Systems 2024/3-ugers kursus/Democrats/SenatorCantwelltweets_clean_PP.csv'

# Contraction map - from https://www.kaggle.com/code/rockystats/topic-modelling-using-nmf#Coherence-Score
c_dict = {
    "ain't": "am not",
    "aren't": "are not",
    "can't": "cannot",
    "can't've": "cannot have",
    "'cause": "because",
    "could've": "could have",
    "couldn't": "could not",
    "couldn't've": "could not have",
    "didn't": "did not",
    "doesn't": "does not",
    "don't": "do not",
    "hadn't": "had not",
    "hadn't've": "had not have",
    "hasn't": "has not",
    "haven't": "have not",
    "he'd": "he would",
    "he'd've": "he would have",
    "he'll": "he will",
    "he'll've": "he will have",
    "he's": "he is",
    "how'd": "how did",
    "how'd'y": "how do you",
    "how'll": "how will",
    "how's": "how is",
    "i'd": "I would",
    "i'd've": "I would have",
    "i'll": "I will",
    "i'll've": "I will have",
    "i'm": "I am",
    "i've": "I have",
    "isn't": "is not",
    "it'd": "it had",
    "it'd've": "it would have",
    "it'll": "it will",
    "it'll've": "it will have",
    "it's": "it is",
    "let's": "let us",
    "ma'am": "madam",
    "mayn't": "may not",
    "might've": "might have",
    "mightn't": "might not",
    "mightn't've": "might not have",
    "must've": "must have",
    "mustn't": "must not",
    "mustn't've": "must not have",
    "needn't": "need not",
    "needn't've": "need not have",
    "o'clock": "of the clock",
    "oughtn't": "ought not",
    "oughtn't've": "ought not have",
    "shan't": "shall not",
    "sha'n't": "shall not",
    "shan't've": "shall not have",
    "she'd": "she would",
    "she'd've": "she would have",
    "she'll": "she will",
    "she'll've": "she will have",
    "she's": "she is",
    "should've": "should have",
    "shouldn't": "should not",
    "shouldn't've": "should not have",
    "so've": "so have",
    "so's": "so is",
    "that'd": "that would",
    "that'd've": "that would have",
    "that's": "that is",
    "there'd": "there had",
    "there'd've": "there would have",
    "there's": "there is",
    "they'd": "they would",
    "they'd've": "they would have",
    "they'll": "they will",
    "they'll've": "they will have",
    "they're": "they are",
    "they've": "they have",
    "to've": "to have",
    "wasn't": "was not",
    "we'd": "we had",
    "we'd've": "we would have",
    "we'll": "we will",
    "we'll've": "we will have",
    "we're": "we are",
    "we've": "we have",
    "weren't": "were not",
    "what'll": "what will",
    "what'll've": "what will have",
    "what're": "what are",
    "what's": "what is",
    "what've": "what have",
    "when's": "when is",
    "when've": "when have",
    "where'd": "where did",
    "where's": "where is",
    "where've": "where have",
    "who'll": "who will",
    "who'll've": "who will have",
    "who's": "who is",
    "who've": "who have",
    "why's": "why is",
    "why've": "why have",
    "will've": "will have",
    "won't": "will not",
    "won't've": "will not have",
    "would've": "would have",
    "wouldn't": "would not",
    "wouldn't've": "would not have",
    "y'all": "you all",
    "y'alls": "you alls",
    "y'all'd": "you all would",
    "y'all'd've": "you all would have",
    "y'all're": "you all are",
    "y'all've": "you all have",
    "you'd": "you had",
    "you'd've": "you would have",
    "you'll": "you you will",
    "you'll've": "you you will have",
    "you're": "you are",
    "you've": "you have"
}
# Compiling the contraction dict
c_re = re.compile('(%s)' % '|'.join(c_dict.keys()))

# List of stop words
add_stop = ['said', 'say', 'like', ' rt ', 'ad', 'just','dont','im','want','amp', 'im', 'weve', 'going', 'ive','theres', 'heres','lets' ]
stop_words = ENGLISH_STOP_WORDS.union(add_stop)


def clean_text(text):

    # Removing unnecessarry data
    text = text.lower()

    text = re.sub(r'http\S+', '', text) # Remove URLs - assistance from ChatGPT
    text = re.sub(r'\[U\+[0-9A-Fa-f]+\]', '', text) # Remove Unicode - assistance from ChatGPT
    text = re.sub(r'[^a-z\s]', '', text) # Remove special characters - assistance from ChatGPT
    text = re.sub(r'\d+', '', text) # Remove number - assistance from ChatGPT

    text = text.replace('\n', ' ').replace('\t', ' ')
    
    emoji_pattern = re.compile( # - assistance from ChatGPT
        "["  
        u"\U0001F600-\U0001F64F"  
        u"\U0001F300-\U0001F5FF"  
        u"\U0001F680-\U0001F6FF"  
        u"\U0001F1E0-\U0001F1FF"  
        u"\U00002500-\U00002BEF"  
        u"\U00002702-\U000027B0"  
        u"\U00002700-\U000027BF"  
        u"\U0001F900-\U0001F9FF"  
        u"\U0001FA70-\U0001FAFF"  
        u"\U0001F680-\U0001F6FF"  
        u"\U000024C2-\U0001F251"  
        "]+",
        flags=re.UNICODE,
    )
    text = emoji_pattern.sub(r'', text) # Remove emojis
    # Tokenize the text and remove stopwords    
    words = text.split()    
    words = [word for word in words if word.lower() not in stop_words]  # Remove stopwords
    # Join the words back into a cleaned sentence    
    text = ' '.join(words) # Strip leading/trailing whitespacereturn 
    text.strip()
    return text

def create_clean_tweet_list(filename_in, filename_out): # Mainly written self, but with guidance from ChatGPT
 
    with open(filename_in, mode='r', encoding='utf-8', errors='ignore') as file_in:
        reader = csv.reader(file_in)
        with open(filename_out, mode='w', encoding='utf-8', newline='') as file_out:
            writer = csv.writer(file_out)
            for row in reader:
                if len(row) > 0:
                    clean_tweet = clean_text(row[1])
                    if clean_tweet:  # Avoid empty tweets
                        writer.writerow([clean_tweet])
       
    return "File cleaned"



tweets_list = create_clean_tweet_list(filename_in, filename_out)


IndexError: list index out of range